In [1]:
!pip install --upgrade geopandas mapclassify folium
!pip install census contextily adjustText 

In [6]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import geopandas as gpd
import matplotlib as mpl
import folium
from folium import plugins
from matplotlib import pyplot as plt
import matplotlib.patheffects as pe
import mapclassify
from census import Census
import contextily as ctx 
from adjustText import adjust_text

plt.rcParams["figure.figsize"] = (10,10) # set this once for a default plot size

In [38]:
nj_bus_terminals = gpd.read_file('Data/Bus_Terminals_of_NJ_Transit.geojson')

In [39]:
nj_counties = gpd.read_file("Data/NJ_Counties_NGJIN_2021.geojson")
nj_counties_webm = nj_counties.to_crs(epsg=4326) # for our Folium map we need our layers to be in Web Mercator

In [40]:
nj_bus_terminals.head()

,OBJECTID,COUNTY,LATITUDE,LONGITUDE,TYPE,OWNER,OPERATOR,PARKING,NAME,MUNICIPALITY,TICKET_SELLER,ORIG_FID,SE_ANNO_CAD_DATA,geometry
0,1,"New York, NY",40.848999,-73.939099,Terminal,Port Authority of NY & NJ,Port Authority of NY & NJ,No,George Washington Bridge,New York,NJ Transit,None,,POINT (-73.93910 40.84901)
1,2,Passaic,40.922456,-74.267799,Park/Ride,NJ DOT,NJ Transit,Yes,Mothers Park & Ride,Wayne Twp,None,None,,POINT (-74.26780 40.92246)
2,3,Hudson,40.731134,-74.062833,Terminal,Port Authority of NY & NJ,Port Authority of NY & NJ,No,Journal Square,Jersey City,None,None,,POINT (-74.06283 40.73114)
3,4,Essex,40.734715,-74.163276,Terminal,NJ Transit,NJ Transit,No,Newark Penn Station,Newark,NJ Transit & Greyhound,None,,POINT (-74.16328 40.73472)
4,5,Essex,40.726428,-74.227144,Terminal,NJ Transit,NJ Transit,Yes,Irvington Bus Terminal,Irvington Twp,Contracted Ticket Agent,None,,POINT (-74.22715 40.72644)


In [41]:
nj_bus_terminals.crs #web mercator check

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [42]:
nj_bus_terminals_list = [[point.xy[1][0], point.xy[0][0]] for point in nj_bus_stops.geometry]


In [49]:
# Create a new Folium map

map = folium.Map(location=[40.18625506916122, -74.55161826002009], tiles=None, zoom_start=9, height = 850, width = 900)
folium.TileLayer('CartoDB positron', name='Carto Positron').add_to(map)

# Create a geometry list from the GeoDataFrame
nj_bus_terminals_list = [[point.xy[1][0], point.xy[0][0]] for point in nj_bus_terminals.geometry]

i = 0 # initialize counter at zero
for coordinates in nj_bus_terminals_list:   # Iterate through list and add a marker for each dispensary, color-coded by type
    if nj_bus_terminals.PARKING[i] == "Yes":
        type_color = "green", 
    else:
        type_color = "red"

    map.add_child( # Place the markers with the popup labels and data
        folium.Marker(location = coordinates,
                      popup = folium.Popup("<b>Name:</b> " + str(nj_bus_terminals.NAME[i]) + "<br><b>Municipality:</b> " + str(nj_bus_terminals.MUNICIPALITY[i]).title() 
                      + "<br><b>Parking:</b> " + str(nj_bus_terminals.PARKING[i]), max_width=len(f"name= {nj_bus_terminals.MUNICIPALITY[i]}")*20), 
                      icon = folium.Icon(color="%s" % type_color),
                      tooltip = str(nj_bus_terminals.NAME[i])))
    i = i + 1

map # call the map to display it

In [46]:
map.save('nj_bus_terminals.html')# to save